In [1]:
import cascade
from cascade import PyPoint
import pandas as pd
import geopandas as gpd
import time
import random
import string

def random_string(length=5):
    # Generate a random string of a given length (default: 5 characters)
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))

In [2]:
gtfs_path = "/home/chingiz/Rust/py_rust/cascade/cascade-bin/files/Chelyabinsk"
pbf_path = "/home/chingiz/Rust/osm/roads_chel.pbf"
departure = 0
duration = 90000
weekday = "monday"

graph = cascade.create_graph(gtfs_path, pbf_path, departure, duration, weekday)

Filtering left 242556 rows
number of edges 232608
1
84371
Graph creation time: 2.382314708s


In [3]:
mapping = graph.get_mapping()
len(mapping)

85728

In [4]:
grid = gpd.read_file(r"/home/chingiz/Rust/py_rust/cascade/cascade-bin/files/chlyab_grid.gpkg")

In [5]:
def gdf_to_pypoint_list(gdf):
    points_list = []
    for _, row in gdf.iterrows():
        point_geom = row['geometry']  # Assumes 'geometry' column has shapely Point objects
        py_point = PyPoint(x=point_geom.x, y=point_geom.y, id=str(row['id']))
        points_list.append(py_point)
    return points_list

In [6]:
list_of_points = gdf_to_pypoint_list(grid)

In [7]:
mapping = {p.id():(p.y(),p.x()) for p in list_of_points}

In [8]:
start = time.perf_counter()
result = cascade.calculate_od_matrix(graph, list_of_points, 43200)
print("Time elapsed", time.perf_counter() - start, len(result))

# crate dataframe from result dict of dict
df: pd.DataFrame = pd.DataFrame.from_dict(result, orient="index").fillna(0)

df_long = df.stack().reset_index()
df_long.columns = ['from_point', 'to_point', 'weight']

Time elapsed 3.4875436769998487 522


In [9]:
df_long["coord_source"] = df_long["from_point"].apply(lambda x: mapping.get(x))
df_long["coord_target"] = df_long["to_point"].apply(lambda x: mapping.get(x))

In [10]:
df_long

,from_point,to_point,weight,coord_source,coord_target
0,196,15,3909.245150,"(55.13270479471254, 61.388540322469645)","(55.13770479471254, 61.33854032246965)"
1,196,402,4634.474789,"(55.13270479471254, 61.388540322469645)","(55.18270479471254, 61.44854032246965)"
2,196,180,1680.678523,"(55.13270479471254, 61.388540322469645)","(55.12270479471254, 61.38354032246965)"
3,196,208,2750.671050,"(55.13270479471254, 61.388540322469645)","(55.16270479471254, 61.39354032246965)"
4,196,264,2476.579636,"(55.13270479471254, 61.388540322469645)","(55.15270479471254, 61.40854032246965)"
...,...,...,...,...,...
272479,345,197,5398.658705,"(55.19770479471254, 61.433540322469646)","(55.12770479471254, 61.388540322469645)"
272480,345,400,1418.689136,"(55.19770479471254, 61.433540322469646)","(55.19270479471254, 61.44854032246965)"
272481,345,219,3061.779748,"(55.19770479471254, 61.433540322469646)","(55.19770479471254, 61.39854032246965)"
272482,345,190,3502.589423,"(55.19770479471254, 61.433540322469646)","(55.16270479471254, 61.388540322469645)"


In [11]:
df2 = df_long.groupby("from_point")["weight"].mean().reset_index()
df2["from_point"] = df2["from_point"].astype(int)

In [12]:
grid = pd.merge(grid, df2, left_on="id", right_on="from_point")

In [14]:
grid.explore(
    column="weight",
    cmap="RdYlGn_r",
    legend=True,
    tooltip="weight",
    legend_kwds=dict(colorbar=False),
    scheme="Quantiles",
    k=6,
)